In [1]:
%%file count_words_bigrams.py
# John Stockton
# CS-351
# Wang
# Winter 2022

from mrjob.job import MRJob

# TODO: Document...
class MRMostUsedWord(MRJob):

    def mapper(self, _, line):
        # Scrub the line for invalid characters
        scrubbedLine = ''
        for char in line: 
            if char.isalpha() or char.isspace():
                scrubbedLine += char.upper()
            else: 
                scrubbedLine += ' '
        # Split our string into individual words
        words = scrubbedLine.split()
        finalIdx = len(words)-1
        for i in range(len(words)-1):
            yield words[i], 1              # yield the current word
            yield (words[i],words[i+1]), 1 # yield current and next words
        yield words[finalIdx], 1

    def combiner(self, key, values):
        yield key, sum(values)

    def reducer(self, key, values):
        yield key, sum(values)

# TODO: Document...
if __name__ == '__main__':
    MRMostUsedWord.run()

Overwriting count_words_bigrams.py


In [2]:
!python conditional_word.py < test.csv > test_out/result.txt

Using configs in /Users/johnstocktoniv/.mrjob.conf
No configs specified for inline runner
Creating temp directory /var/folders/5s/hq_d6y217wb9tt3p5v4_25cw0000gn/T/conditional_word.johnstocktoniv.20220309.140648.522013
Running step 1 of 1...
reading from STDIN
job output is in /var/folders/5s/hq_d6y217wb9tt3p5v4_25cw0000gn/T/conditional_word.johnstocktoniv.20220309.140648.522013/output
Streaming final output from /var/folders/5s/hq_d6y217wb9tt3p5v4_25cw0000gn/T/conditional_word.johnstocktoniv.20220309.140648.522013/output...
Removing temp directory /var/folders/5s/hq_d6y217wb9tt3p5v4_25cw0000gn/T/conditional_word.johnstocktoniv.20220309.140648.522013...


In [3]:
%%file cond_prob.py
# John Stockton
# CS-351
# Wang
# Winter 2022

from sys import argv
import re

# TODO: Document...
class node:
    def __init__(self,word=None,count=None,next=None):
        self.word  = word
        self.count = count
        self.next  = next
    def set_data(self,word,count):
        self.word = word
        self.count = count
    def set_next(self,node):
        self.next = node
    def get_count(self,word):
        curr = self
        while curr.next!=None:
            if word==curr.word:
                return curr.count
            curr = curr.next
        return 0

# TODO: Document...
def main(inputName, outputName):
    inFile = open(inputName, 'r')
    outFile = open(outputName, 'w+')

    lines = inFile.readlines()
    words = []
    bigrams = []
    for line in lines:
        tokens = re.findall('[a-zA-Z0-9]+', line)
        if len(tokens)==2:
            words += [tokens]
        else:
            bigrams += [tokens]

    list = node()
    curr = list
    for word in words:
        curr.set_data(word[0],int(word[1]))
        curr.set_next(node())
        curr = curr.next

    for bigram in bigrams:
        prob = int(bigram[2])/list.get_count(bigram[0])
        prob = round(prob,3)
        bigram = f"P({bigram[1]} | {bigram[0]}) ~= {prob}\n"
        outFile.write(bigram)
        
# TODO: Document...
if __name__=='__main__':
    main(argv[1],argv[2])

Overwriting cond_prob.py


In [4]:
!python cond_prob.py test_out/result.txt test_out/conditional.txt

In [5]:
%%file top_words.py
# John Stockton
# CS-351
# Wang
# Winter 2022

from sys import argv
import re

def find_max(words):
    num_words = int(len(words)/2)
    max = "",0
    maxIdx = 0
    for i in range(num_words):
        curr_word = words[2*i]
        curr_word_count = int(words[2*i+1])
        if int(curr_word_count)>max[1]:
            max = curr_word,curr_word_count
            maxIdx = 2*i
    # We found the max, delete it from list
    words[maxIdx] = ""
    words[maxIdx+1] = 0
    return max

def get_top_words(words, mys, num):
    top_words = []
    for i in range(0,num):
        top_words += find_max(words)
        curr_count = int(top_words[2*i+1])
        top_words[2*i+1] = round(curr_count/mys,3)
    return top_words

# TODO: Document...
def main(inputName, outputName):
    inFile = open(inputName, 'r')
    outFile = open(outputName, 'w+')

    lines = inFile.readlines()
    num_my = 0
    after_my = []
    for line in lines:
        tokens = re.findall('[a-zA-Z0-9]+', line)
        if tokens[0]=='MY':
            if len(tokens)==2:
                num_my = int(tokens[1])
            else:
                after_my += [tokens[1], tokens[2]]
        
    if after_my==0:
        outFile.write('\'My\' does not occur in the input\n')
    elif int(len(after_my)/2)>=10:
        top_words = get_top_words(after_my, num_my, 10)
    else:
        top_words = get_top_words(after_my,num_my,int(len(after_my)/2))
    
    for i in range(0,int(len(top_words)/2)):
        curr_word = top_words[2*i]
        curr_prob = top_words[2*i+1]
        outFile.write(f'{int(len(top_words)/2)-i}. {curr_word} - {curr_prob}\n')
        
# TODO: Document...
if __name__=='__main__':
    main(argv[1],argv[2])

Overwriting top_words.py


In [6]:
!python top_words.py test_out/result.txt test_out/topWords.txt